In [2]:
from bs4 import BeautifulSoup
import requests
import re
import json
import pandas as pd

In [48]:
url = "https://www.millenaire3.com/ressources/2023/changer-les-comportements-revue-et-enseignements-des-actions-metropolitaines"
url2 = "https://www.millenaire3.com/ressources/2023/agir-sur-les-modes-de-vie-une-nouvelle-grille-d-analyse"
url3 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire"
url4 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/institut-transitions-notre-but-c-est-de-former-ceux-qui-peuvent-agir-des-aujourd-hui"
url5 = "https://www.millenaire3.com/dossiers/2022/resilience-quelles-articulations-entre-acteurs-publics-et-initiatives-du-territoire/les-alchimistes-lyon-alimentation-gaspillage-compostage-aucun-de-ces-sujets-ne-laisse-indifferent"
url6 = "https://www.millenaire3.com/ressources/2023/crise-ecologique-le-metabolisme-de-l-economie-au-caeur-des-enjeux"
url7 = "https://www.millenaire3.com/dossiers/2015/la-soierie-lyonnaise-entre-tradition-et-innovation-archives/la-soie-a-lyon-une-initiative-du-pouvoir-royal"
urls = [url, url2, url3, url4, url5, url6, url7]

with open("./M3urls.txt", "r") as f:
        urls = f.read().split("\n")

In [59]:
def try_another_way(url, resume):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    article = soup.find_all("div",  {"class" : "ezrichtext-field"})
    txt = []
    for i, art in enumerate(article):
        if art.text != resume:
    #         p = art.find_all("p")
    #         for para in p:
    #             txt.append(re.sub("<strong>|<\\\strong>|<p>|</p>|<\\\p>|</strong>|<em>|</em>", "", str(para).replace("<br/>", " ")))
    #             # print(para.text.replace("\n", " "))

    # txt = "".join(txt)
            txt = art.text
    return(txt)

In [163]:
def scrap_page(url, container = {}, mult_items = {}, errors = {}):
    
    html = requests.get(url)
    if(html.status_code) != 200:
        print(url, html.status_code)
        errors["deadlink"].append(url)
        return(1)
    soup = BeautifulSoup(html.text, "html.parser")

    #TYPE
    type_liste = soup.find_all("p", {"class" : re.compile("^over-under")})
    if len(type_liste) > 0:
        type = ""
        for typ in type_liste:
            if re.search("Étude|Dossier|Article|Texte", typ.text) == None:
                errors["TYPE"].append(url)
            else:
                if re.search("Interview|Texte", typ.text) == None:
                    type = typ.text.replace("\n", "").replace("\t", "").replace(" ", "")
                elif re.search("Interview", typ.text) == None:
                    type = "Texte"
                    itw_de = typ.text.replace("\n", "").replace("\t", "")
                else:
                    type = "Interview"
                    itw_de = typ.text.replace("\n", "").replace("\t", "")
    else:
        return(1)
    if len(type) > 0:
        container["TYPE"].append(type)
    else:
        errors["TYPE"].append(url)
        return(1)

    #RESUME
    if type == "Dossier":
        resume = soup.find("div", {"class" : "eztext-field"})
    elif type == "Interview":
        resume = soup.find("div", {"class" : "ezrichtext-field"})
    else: 
        resume = soup.find("div", {"class" : "eztext-field"})
    if len(resume) > 0:
        container["RESUME"].append(resume.text)
    else:
        errors["RESUME"].append(url)
        container["RESUME"].append("NA")

    #ARTICLE
    if type != "Interview":
        try:
            article = soup.find("div",  {"class" : "ezrichtext-field"}).text
        except:
            article = ""
    else:
        lines = soup.find_all("div", {"class" : "interview-item"})
        print(lines)
        if len(lines) == 0:
            art = soup.find_all("div",  {"class" : "ezrichtext-field"})
            print(art)
            for info in art:
                if info != resume.text:
                    article = info.text
        else:
            article = ""
            for i, line in enumerate(lines):
                article += line.find("p", {"class" : "over-under-line"}).text + " "
                temp = line.find_all("div", {"class" : "ezrichtext-field"})
                for info in temp:
                    article += info.find("p").text
    if len(article) > 0:
        container["ARTICLE"].append(article.replace("\n", " "))
    else:
        # article = try_another_way(url, resume.text)
        # if len(article) > 0:
        #     container["ARTICLE"].append(article)
        # else:
        errors["ARTICLE"].append(url)
        container["ARTICLE"].append("NA")

    #TITRE
    titre = soup.find("h1", {"class": "uppercase h1-special"}).contents[0]
    titre = titre.replace("\n", "").replace("\t", "").strip()
    if len(titre) > 0:
        container["TITRE"].append(titre)
    else:
        errors["TITRE"].append(url)
        container["TITRE"].append("NA")
    
    #ID - in db
    try:
        ID = soup.find("article", {"class" : "sidebar-block-classic sidebar-block-download"})
        ID = ID.a["href"].split("/")
        ID = ID[len(ID)-1]
        container["IDinDB"].append(ID)
    except:
        container["IDinDB"].append("NA")

    #DATE
    date = soup.find("span", {"class" : "publication_date"}).text
    if len(date) > 0:
        container["DATE"].append(date)
    else:
        errors["DATE"].append(url)
        container["DATE"].append("NA")

    mult_items[url] = {"auteurs" : [], "tags" : [], "liens" : []}

    #TAGS
    tags_liste = soup.find_all("ul", {"class": "list-inline keywords"})
    tags = []
    for tag in tags_liste:
        a = tag.find_all("a")
        for tag in a:
            if re.search("tag", tag["href"]) != None:
                tags += tag.contents
    if len(tags) > 0:
        mult_items[url]["tags"] = tags
    else:
        mult_items[url]["tags"] = "NA"

    #AUTHORS
    if type == "Interview":
        liste = soup.find_all("ul", {"class": "list-inline keywords"})
        authors = []
        for elem in liste:
            a = elem.find_all("a")
            for author in a:
                if re.search("auteurs", author["href"]) != None:
                    authors.append(author.contents[0].strip())
    else:
        auth = soup.find_all("span", {"class" : "author-name"})
        authors = []
        for author in auth:
            author = repr(author.text).replace("\\xa0", " ").replace("\\t", "").replace("\\n", "").replace("'", "")
            if author not in authors:
                authors.append(author)
    if len(authors) > 0:
        mult_items[url]["auteurs"] = authors
    else:
        mult_items[url]["auteurs"] = "NA"
        
    #Liens
    page_links = soup.find_all("h3", {"class" : "h3 uppercase"})
    links = []
    for link in page_links:
        links.append("https://www.millenaire3.com"+link.a["href"])
    if len(links) > 0:
        mult_items[url]["liens"] = links
    else:
        mult_items[url]["liens"] = "NA"

    #print(url)
    container["URL"].append(url)

    return(container, mult_items, errors)


In [164]:
container = {"URL" : [], 
             "TITRE" : [], 
             "IDinDB" : [], 
             "TYPE" : [], 
             "RESUME" : [], 
             "ARTICLE" : [],
             "DATE" : []}

mult_items = {}

errors = {
    "TITRE" : [],
    "TYPE" : [],
    "RESUME" : [],
    "ARTICLE" : [],
    "DATE" : [],
    "deadlink" : []
}

for i, url in enumerate(urls):
    print(i, url)
    if re.search("/auteurs/|/tag/|partage_email", url) == None:
        scrap_page(url, container, mult_items, errors)
    else:
        print("not an article")
    # except:
    #     print("Not an article")

0 https://www.millenaire3.com/generated_pdf/38168
https://www.millenaire3.com/generated_pdf/38168 404
1 https://www.millenaire3.com/generated_pdf/43855
https://www.millenaire3.com/generated_pdf/43855 404
2 https://www.millenaire3.com/partage_email/33794
not an article
3 https://www.millenaire3.com/partage_email/40666
not an article
4 https://www.millenaire3.com/generated_pdf/37304
https://www.millenaire3.com/generated_pdf/37304 404
5 https://www.millenaire3.com/auteurs/oullier-olivier
not an article
6 https://www.millenaire3.com/dossiers/2020/covid-19-relectures-de-corpus-tous-aux-archives
7 https://www.millenaire3.com/Interview/2021/joel-de-rosnay-resilience-et-sagesse-sont-les-mots-cles-pour-pouvoir-envisager-le-futur
8 https://www.millenaire3.com/Interview/2019/entrepreneur-innovant-social-et-solidaire-l-exemple-du-grap
9 https://www.millenaire3.com/dossiers/2015/attractivite-de-la-metropole-et-societe-du-temps-libre-archives/Developper-l-economie-de-proximite-3.-Le-ressort-de-la-ca

AttributeError: 'NoneType' object has no attribute 'text'

In [161]:
for keys in container:
    print(keys, ":", len(container[keys]))

URL : 4
TITRE : 4
IDinDB : 4
TYPE : 4
RESUME : 4
ARTICLE : 4
DATE : 4


In [162]:
pd.DataFrame(container)

,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE
0,https://www.millenaire3.com/dossiers/2022/cycl...,"Cycle Veille M3 / Gastronomie : manger, partag...",NA,Dossier,Au cœur d’une métropole dont la ville-centre p...,"Sommaire :Gouvernance, politiques publiques : ...",24/08/2022
1,https://www.millenaire3.com/dossiers/2019/limi...,Limites planétaires 11/11 : Nouvelles pollutio...,491664,Étude,"Parmi les pollutions chimiques, il existe de n...",\n\n© La Métropole de Lyon - Skoli\n\n Retrou...,02/07/2019
2,https://www.millenaire3.com/dossiers/l-eau-3-3...,Eau futurE / L’eau et nous demain : panorama d...,531437,Étude,L’eau est un bien commun. Tout ce qu’on fait a...,Sommaire : Préambule : explorer les imaginaire...,06/12/2022
3,https://www.millenaire3.com/ressources/2021/ve...,Veille M3 / Le monde du sport peut-il encore g...,NA,Article,"En novembre 2015, alors que les négociations d...",Le sport menacé par le changement climatique A...,06/07/2021


In [140]:
print(json.dumps(container, indent = 3, ensure_ascii=False))
print(json.dumps(mult_items, indent = 3, ensure_ascii=False))

{
   "URL": [
      "https://www.millenaire3.com/dossiers/2020/veille-m3/Veille-M3-Les-nouvelles-pratiques-sportives-sous-l-empire-des-chiffres",
      "https://www.millenaire3.com/Interview/2004/les-enjeux-pour-un-laboratoire-en-sante-animale-comme-merial",
      "https://www.millenaire3.com/Interview/2020/barbara-jacques-recruter-des-competences-dans-les-industries-de-la-sante",
      "https://www.millenaire3.com/dossiers/Covid-19-Relectures-de-corpus-Tous-aux-archives",
      "https://www.millenaire3.com/dossiers/2021/economie-circulaire-au-dela-du-recyclage-comment-transformer-l-economie/L-economie-circulaire",
      "https://www.millenaire3.com/dossiers/2022/economie-regenerative-quel-chemin-pour-reconcilier-activites-humaines-et-cycles-naturels/ingenierie-et-restauration-ecologiques-la-biodiversite-est-elle-reparable",
      "https://www.millenaire3.com/dossiers/2015/la-soierie-lyonnaise-entre-tradition-et-innovation-archives/la-soie-a-lyon-une-initiative-du-pouvoir-royal",
     

In [103]:
i = 0
for URL in urls:
    if URL not in container["URL"] and URL not in errors["deadlink"] and "partage_email" not in URL and "/auteurs/" not in URL and "/tag/" not in URL:
        i += 1
        print(i, URL)

1 https://www.millenaire3.com/donnees-personnelles
2 https://www.millenaire3.com/qui-sommes-nous
3 https://www.millenaire3.com/
4 https://www.millenaire3.com/tous-les-dossiers
5 https://www.millenaire3.com/modes-d-action
6 https://www.millenaire3.com/Accessibilite
7 https://www.millenaire3.com/tous-les-auteurs?l=K
8 https://www.millenaire3.com/tous-les-auteurs?l=A
9 https://www.millenaire3.com/tous-les-auteurs?l=S
10 https://www.millenaire3.com/Newsletter/Focus-Janvier-2021
11 https://www.millenaire3.com/tous-les-auteurs?l=D
12 https://www.millenaire3.com/media/videos/comment-les-friches-sont-devenues-les-temples-de-la-hype
13 https://www.millenaire3.com/dossiers/2021/quelles-modalites-pour-comprendre-l-usager-citoyen/Corinne-Hooge-En-matiere-d-evaluation-la-satisfaction-des-usagers-est-une-brique-qui-doit-etre-croisee-avec-d-autres-angles-d-analyse
14 https://www.millenaire3.com/tous-les-auteurs?l=T
15 https://www.millenaire3.com/media/veillem3/veille-sport3-l-impact-des-stations-de-s

In [111]:
#scrap_page("https://www.millenaire3.com/Interview/2004/les-enjeux-pour-un-laboratoire-en-sante-animale-comme-merial", container, mult_items, errors)
errors

{'TITRE': [],
 'TYPE': [],
 'RESUME': ['https://www.millenaire3.com/ressources/vivre-et-imaginer-la-ville-entre-cite-et-metropole',
  'https://www.millenaire3.com/ressources/l-integration-de-l-animal-familier-au-sein-des-institutions-pour-personnes-agees',
  'https://www.millenaire3.com/ressources/diversite-culturelle-religion-et-modernite-l-experience-de-la-france-contemporaine',
  'https://www.millenaire3.com/ressources/les-tsiganes-de-hongrie-et-les-politiques-d-integration',
  'https://www.millenaire3.com/ressources/forces-spirituelles-une-dette-a-l-egard-de-l-humanite',
  'https://www.millenaire3.com/ressources/vers-un-developpement-durable-de-qualite-a-quelles-conditions-peut-on-agir',
  'https://www.millenaire3.com/ressources/consultation-sur-la-condition-penitentiaire',
  'https://www.millenaire3.com/ressources/la-situation-des-personnes-agees-immigrees-dans-le-contexte-urbain-suisse-l-exemple-de-geneve',
  'https://www.millenaire3.com/ressources/lyon-2020-synthese-du-groupe-de

In [109]:
for i, URL in enumerate(container["URL"]):
    if URL == "https://www.millenaire3.com/ressources/vivre-et-imaginer-la-ville-entre-cite-et-metropole":
        print(container["RESUME"][i])

NA
